# Create Boundary Conditions from NAQFC for RRFS-CMAQ

To use:
 1) First load the modules needed 
 2) Fix a RRFS met boundary condition file (this is a dummy file just to get the location and halo layers)
 3) Create emissions for all months

### Step 1

In [ ]:
import regrid_naqfc_to_ufs as rnu
import xarray as xr 
from numpy import arange
import os

### Step 2

In [ ]:
rrfs_boundary = 'gfs_bndy.tile7.000.nc' # a RRFS MET BNDY FILE
q = xr.open_dataset(rrfs_boundary)
drop_vars = [n for n in q.data_vars if 'o3mr' not in n] # drops all variables except o3mr to create a dummy variable
q.drop(drop_vars).rename({'o3mr_bottom':'o3_bottom','o3mr_top':'o3_top','o3mr_left':'o3_left','o3mr_right':'o3_right'}).to_netcdf('dummy_bndy.nc')

### Step 3

In [ ]:
months = [str(i).zfill(2) for i in arange(1,13)]
output_path = 'lbcs_13_bndy'
# check if path exists
if not os.path.isdir(output_path): 
    os.makedirs(output_path)
for m in months:
    print('Processing month... ',m)
    boundary_file = 'dummy_bndy.nc' # edit this to match the output in step 2
    grid_file = 'lbcs_13/grid_spec.nc' # point to the grid_spec.nc from RRFS
    naqfc_bndy_file = 'aqm_conus_12km_geos_2006{}_static_35L.ncf'.format(m) # NAQFC Boundary Condition
    d = rnu.fix_all(boundary_file, naqfc_bndy_file, grid_fname=grid_file)
    output_filename = os.path.join(output_path, 'gfs_bndy_chem_{}.tile7.000.nc'.format(m))
    print('  Outputing to:',output_filename)
    d.to_netcdf(output_filename)

